# Imports (compatible for Python 2 and Python 3)

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

Sort data by start_station_id, tripduration only checking trips with duration <= 3 hours(10800 seconds)

In [3]:
query = '''SELECT * FROM fb55.citibike
WHERE tripduration< 10800
ORDER BY start_station_id ASC, tripduration ASC; '''
table = get_data(query)
table.head()

the_geom start_station_name  the_geom_webmercator  field_1  \
0       NaN   W 52 St & 11 Ave                   NaN     9059   
1       NaN   W 52 St & 11 Ave                   NaN    40772   
2       NaN   W 52 St & 11 Ave                   NaN    40770   
3       NaN   W 52 St & 11 Ave                   NaN    22002   
4       NaN   W 52 St & 11 Ave                   NaN    43107   

   start_station_longitude  tripduration               starttime  \
0               -73.993929           107  2015-02-03 10:10:00+00   
1               -73.993929           189  2015-02-07 10:40:00+00   
2               -73.993929           229  2015-02-07 10:40:00+00   
3               -73.993929           248  2015-02-05 06:19:00+00   
4               -73.993929           252  2015-02-07 15:43:00+00   

                 stoptime  start_station_id  cartodb_id  \
0  2015-02-03 10:12:00+00                72        8998   
1  2015-02-07 10:43:00+00                72       40759   
2  2015-02-07 10:44:00+00                72       40757   
3  2015-02-05 06:24:00+00                72       21948   
4  2015-02-07 15:47:00+00                72       43099   

   start_station_latitude  end_station_id  end_station_name  \
0               40.767272              72  W 52 St & 11 Ave   
1               40.767272              72  W 52 St & 11 Ave   
2               40.767272              72  W 52 St & 11 Ave   
3               40.767272             478  11 Ave & W 41 St   
4               40.767272             515  W 43 St & 10 Ave   

   end_station_latitude  end_station_longitude  bikeid    usertype  \
0             40.767272             -73.993929   19039  Subscriber   
1             40.767272             -73.993929   18131  Subscriber   
2             40.767272             -73.993929   19729  Subscriber   
3             40.760301             -73.998842   16879  Subscriber   
4             40.760094             -73.994618   15277  Subscriber   

   birth_year  gender  
0      1980.0       2  
1      1959.0       2  
2      1995.0       1  
3      1970.0       1  
4      1983.0       1

Only show the top/last 10 records

In [4]:
query = '''SELECT * FROM fb55.citibike
ORDER BY cartodb_id ASC
LIMIT 10; '''
table = get_data(query)
table

the_geom                 start_station_name  the_geom_webmercator  field_1  \
0       NaN                    8 Ave & W 31 St                   NaN       70   
1       NaN                 E 17 St & Broadway                   NaN       71   
2       NaN  Grand Army Plaza & Central Park S                   NaN       72   
3       NaN                  6 Ave & Broome St                   NaN       73   
4       NaN        Lawrence St & Willoughby St                   NaN       74   
5       NaN       Willoughby Ave & Walworth St                   NaN       75   
6       NaN                    W 56 St & 6 Ave                   NaN       76   
7       NaN                     E 4 St & 2 Ave                   NaN       77   
8       NaN           Washington Pl & Broadway                   NaN       78   
9       NaN              Greenwich Ave & 8 Ave                   NaN       79   

   start_station_longitude  tripduration               starttime  \
0               -73.994811           801  2015-02-01 00:00:00+00   
1               -73.990093           379  2015-02-01 00:00:00+00   
2               -73.973715          2474  2015-02-01 00:01:00+00   
3               -74.004704           818  2015-02-01 00:01:00+00   
4               -73.986317           544  2015-02-01 00:01:00+00   
5               -73.953820           717  2015-02-01 00:02:00+00   
6               -73.977225          1306  2015-02-01 00:04:00+00   
7               -73.989780           913  2015-02-01 00:04:00+00   
8               -73.994046           759  2015-02-01 00:04:00+00   
9               -74.002638           585  2015-02-01 00:05:00+00   

                 stoptime  start_station_id  cartodb_id  \
0  2015-02-01 00:14:00+00               521           1   
1  2015-02-01 00:07:00+00               497           2   
2  2015-02-01 00:42:00+00               281           3   
3  2015-02-01 00:15:00+00              2004           4   
4  2015-02-01 00:10:00+00               323           5   
5  2015-02-01 00:14:00+00               373           6   
6  2015-02-01 00:26:00+00               352           7   
7  2015-02-01 00:19:00+00               439           8   
8  2015-02-01 00:17:00+00               335           9   
9  2015-02-01 00:15:00+00               284          10   

   start_station_latitude  end_station_id               end_station_name  \
0               40.750450             423                W 54 St & 9 Ave   
1               40.737050             504                1 Ave & E 15 St   
2               40.764397             127          Barrow St & Hudson St   
3               40.724399             505                6 Ave & W 33 St   
4               40.692362              83  Atlantic Ave & Fort Greene Pl   
5               40.693317            2002   Wythe Ave & Metropolitan Ave   
6               40.763406             504                1 Ave & E 15 St   
7               40.726281             116                W 17 St & 8 Ave   
8               40.729039            2012                E 27 St & 1 Ave   
9               40.739017             444             Broadway & W 24 St   

   end_station_latitude  end_station_longitude  bikeid    usertype  \
0             40.765849             -73.986905   17131  Subscriber   
1             40.732219             -73.981656   21289  Subscriber   
2             40.731724             -74.006744   18903  Subscriber   
3             40.749013             -73.988484   21044  Subscriber   
4             40.683826             -73.976323   19868  Subscriber   
5             40.716887             -73.963198   15854  Subscriber   
6             40.732219             -73.981656   15173  Subscriber   
7             40.741776             -74.001497   17862  Subscriber   
8             40.739445             -73.976806   21183  Subscriber   
9             40.742354             -73.989151   14843  Subscriber   

   birth_year  gender  
0        1978       2  
1        1993       1  
2        1969       2  
3       

List all unique start_station_id values

In [5]:
query = '''SELECT DISTINCT(start_station_id) FROM fb55.citibike; '''
table = get_data(query)
table.head()

start_station_id  Unnamed: 1
0               120         NaN
1               285         NaN
2               251         NaN
3               195         NaN
4               453         NaN

Count the number of trips

In [6]:
query = '''SELECT COUNT(cartodb_id) FROM fb55.citibike
WHERE tripduration>0; '''
table = get_data(query)
table

count  Unnamed: 1
0  46200         NaN

Find the average/min/max trip duration

In [7]:
query = '''SELECT AVG(tripduration), MAX(tripduration), MIN(tripduration)  FROM fb55.citibike; '''
table = get_data(query)
table

avg    max  min
0  675.865823  43016   60

Selecting trips started on Feb-02-2015 only 

In [8]:
query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(DAY from starttime)=2 AND EXTRACT(MONTH from starttime)=2 AND EXTRACT(YEAR from starttime)=2015; '''
table = get_data(query)
table

the_geom           start_station_name  the_geom_webmercator  field_1  \
0          NaN              5 Ave & E 29 St                   NaN     7433   
1          NaN         Allen St & Hester St                   NaN     7779   
2          NaN            E 5 St & Avenue C                   NaN       11   
3          NaN              9 Ave & W 16 St                   NaN      627   
4          NaN              9 Ave & W 16 St                   NaN      991   
5          NaN              W 33 St & 7 Ave                   NaN     1037   
6          NaN              W 27 St & 7 Ave                   NaN     6510   
7          NaN              E 11 St & 1 Ave                   NaN     6511   
8          NaN         Peck Slip & Front St                   NaN     6512   
9          NaN              E 16 St & 5 Ave                   NaN     6513   
10         NaN              W 41 St & 8 Ave                   NaN     6514   
11         NaN              Dean St & 4 Ave                   NaN     6515   
12         NaN              9 Ave & W 18 St                   NaN     6516   
13         NaN          Reade St & Broadway                   NaN     6517   
14         NaN              8 Ave & W 31 St                   NaN     6518   
15         NaN              1 Ave & E 15 St                   NaN     6519   
16         NaN           Broadway & E 14 St                   NaN     6520   
17         NaN            E 2 St & Avenue B                   NaN     6521   
18         NaN         Peck Slip & Front St                   NaN     6522   
19         NaN   Columbia St & Rivington St                   NaN     6523   
20         NaN              W 20 St & 8 Ave                   NaN     6524   
21         NaN          St Marks Pl & 2 Ave                   NaN     6525   
22         NaN              W 20 St & 8 Ave                   NaN     6526   
23         NaN              W 37 St & 5 Ave                   NaN     6527   
24         NaN              W 13 St & 6 Ave                   NaN     6528   
25         NaN             10 Ave & W 28 St                   NaN     6529   
26         NaN              E 25 St & 2 Ave                   NaN     6530   
27         NaN             W 4 St & 7 Ave S                   NaN     6531   
28         NaN              W 20 St & 7 Ave                   NaN     6532   
29         NaN             W 46 St & 11 Ave                   NaN     6533   
...        ...                          ...                   ...      ...   
1429       NaN      Allen St & E Houston St                   NaN     7938   
1430       NaN           Broadway & W 29 St                   NaN     7939   
1431       NaN              W 56 St & 6 Ave                   NaN     7940   
1432       NaN             10 Ave & W 28 St                   NaN     7941   
1433       NaN              W 26 St & 8 Ave                   NaN     7942   
1434       NaN           Broadway & E 22 St                   NaN     7943   
1435       NaN              E 47 St & 2 Ave                   NaN     7944   
1436       NaN           Broadway & W 32 St                   NaN     7945   
1437       NaN              8 Ave & W 31 St                   NaN     7946   
1438       NaN              9 Ave & W 18 St                   NaN     7947   
1439       NaN        LaGuardia Pl & W 3 St                   NaN     7948   
1440       NaN              W 26 St & 8 Ave                   NaN     7949   
1441       NaN         Cliff St & Fulton St                   NaN     7950   
1442       NaN      Rivington St & Ridge St                   NaN     7951   
1443       NaN              W 49 St & 8 Ave                   NaN     7952   
1444       NaN        Pershing Square North                   NaN     7953   
1445       NaN         E 30 St & Park Ave S                   NaN     7954   
1446       NaN  Sullivan St & Washington Sq                   NaN     7955   
1447       NaN              W 42 St & 8 Ave                   NaN     7956   
1448       NaN           Bro

Selecting trips started on the weekends

In [9]:
query = '''SELECT * FROM fb55.citibike
WHERE  EXTRACT(DOW from starttime)=6 OR EXTRACT(DOW from starttime)=0; '''
table = get_data(query)
table

the_geom        start_station_name  the_geom_webmercator  field_1  \
0           NaN        E 20 St & Park Ave                   NaN      175   
1           NaN          W 43 St & 10 Ave                   NaN     1159   
2           NaN         E 6 St & Avenue B                   NaN     2827   
3           NaN           E 12 St & 3 Ave                   NaN     4961   
4           NaN           W 41 St & 8 Ave                   NaN     6156   
5           NaN           E 20 St & 2 Ave                   NaN    39894   
6           NaN        Broadway & W 60 St                   NaN    41512   
7           NaN         E 6 St & Avenue D                   NaN    41595   
8           NaN            Great Jones St                   NaN    41806   
9           NaN           W 13 St & 6 Ave                   NaN    42316   
10          NaN          W 34 St & 11 Ave                   NaN    42532   
11          NaN           W 41 St & 8 Ave                   NaN    42915   
12          NaN           E 25 St & 1 Ave                   NaN    44448   
13          NaN           E 15 St & 3 Ave                   NaN    44744   
14          NaN           W 33 St & 7 Ave                   NaN    44834   
15          NaN           E 33 St & 2 Ave                   NaN    45301   
16          NaN           8 Ave & W 33 St                   NaN        0   
17          NaN        Broadway & W 39 St                   NaN        1   
18          NaN           W 18 St & 6 Ave                   NaN        2   
19          NaN        Broadway & E 22 St                   NaN        3   
20          NaN          W 46 St & 11 Ave                   NaN        4   
21          NaN            Great Jones St                   NaN        5   
22          NaN        Carmine St & 6 Ave                   NaN        6   
23          NaN           E 12 St & 3 Ave                   NaN        7   
24          NaN           W 20 St & 7 Ave                   NaN        8   
25          NaN           E 11 St & 1 Ave                   NaN        9   
26          NaN           W 24 St & 7 Ave                   NaN       10   
27          NaN           W 33 St & 7 Ave                   NaN       38   
28          NaN           W 17 St & 8 Ave                   NaN       49   
29          NaN           E 15 St & 3 Ave                   NaN       50   
...         ...                       ...                   ...      ...   
13201       NaN  Washington Pl & Broadway                   NaN    46170   
13202       NaN      Allen St & Hester St                   NaN    46171   
13203       NaN     Warren St & Church St                   NaN    46172   
13204       NaN           E 12 St & 3 Ave                   NaN    46173   
13205       NaN           Dean St & 4 Ave                   NaN    46174   
13206       NaN      Allen St & Hester St                   NaN    46175   
13207       NaN        Broadway & E 22 St                   NaN    46176   
13208       NaN     Washington Pl & 6 Ave                   NaN    46177   
13209       NaN           E 31 St & 3 Ave                   NaN    46178   
13210       NaN        Broadway & W 60 St                   NaN    46179   
13211       NaN        Carmine St & 6 Ave                   NaN    46180   
13212       NaN           W 52 St & 5 Ave                   NaN    46181   
13213       NaN           Dean St & 4 Ave                   NaN    46182   
13214       NaN           9 Ave & W 45 St                   NaN    46183   
13215       NaN        Broadway & W 58 St                   NaN    46184   
13216       NaN        Broadway & W 58 St                   NaN    46185   
13217       NaN   Watts St & Greenwich St                   NaN    46186   
13218       NaN           W 51 St & 6 Ave                   NaN    46187   
13219       NaN   University Pl & E 14 St                   NaN    46188   
13220       NaN           W 11 St & 6 Ave                   NaN    46189   
13221       NaN           W 17 St & 8 Ave              

What are average trip duration during weekends? 

In [10]:
query = '''SELECT AVG(tripduration) FROM fb55.citibike
WHERE  EXTRACT(DOW from starttime)=6 OR EXTRACT(DOW from starttime)=0; '''
table = get_data(query)
table

avg  Unnamed: 1
0  662.942181         NaN

Selecting trips started on the weekdays

In [11]:
query = '''SELECT * FROM fb55.citibike
WHERE NOT (EXTRACT(DOW from starttime)=6 OR EXTRACT(DOW from starttime)=0); '''
table = get_data(query)
table

the_geom              start_station_name  the_geom_webmercator  \
0           NaN                 5 Ave & E 29 St                   NaN   
1           NaN            Allen St & Hester St                   NaN   
2           NaN                 E 39 St & 3 Ave                   NaN   
3           NaN                10 Ave & W 28 St                   NaN   
4           NaN                  E 2 St & 2 Ave                   NaN   
5           NaN                 1 Ave & E 44 St                   NaN   
6           NaN                 W 45 St & 6 Ave                   NaN   
7           NaN                 W 18 St & 6 Ave                   NaN   
8           NaN                 E 58 St & 3 Ave                   NaN   
9           NaN                 E 33 St & 5 Ave                   NaN   
10          NaN                 W 31 St & 7 Ave                   NaN   
11          NaN                 5 Ave & E 29 St                   NaN   
12          NaN                 E 39 St & 2 Ave                   NaN   
13          NaN                10 Ave & W 28 St                   NaN   
14          NaN                 W 42 St & 8 Ave                   NaN   
15          NaN                 E 27 St & 1 Ave                   NaN   
16          NaN                  Great Jones St                   NaN   
17          NaN                 W 49 St & 8 Ave                   NaN   
18          NaN              Broadway & W 24 St                   NaN   
19          NaN                 E 40 St & 5 Ave                   NaN   
20          NaN                 E 16 St & 5 Ave                   NaN   
21          NaN              Broadway & W 36 St                   NaN   
22          NaN                 8 Ave & W 31 St                   NaN   
23          NaN                 W 45 St & 6 Ave                   NaN   
24          NaN             FDR Drive & E 35 St                   NaN   
25          NaN                 6 Ave & W 33 St                   NaN   
26          NaN                 E 27 St & 1 Ave                   NaN   
27          NaN                12 Ave & W 40 St                   NaN   
28          NaN                 W 52 St & 9 Ave                   NaN   
29          NaN                 8 Ave & W 33 St                   NaN   
...         ...                             ...                   ...   
32939       NaN          W Broadway & Spring St                   NaN   
32940       NaN              Broadway & W 41 St                   NaN   
32941       NaN                 W 38 St & 8 Ave                   NaN   
32942       NaN                 9 Ave & W 22 St                   NaN   
32943       NaN  Lafayette Ave & Fort Greene Pl                   NaN   
32944       NaN              Broadway & E 22 St                   NaN   
32945       NaN                 E 23 St & 1 Ave                   NaN   
32946       NaN                 E 19 St & 3 Ave                   NaN   
32947       NaN              Broadway & E 22 St                   NaN   
32948       NaN      Greenwich Ave & Charles St                   NaN   
32949       NaN            Allen St & Hester St                   NaN   
32950       NaN              E 20 St & Park Ave                   NaN   
32951       NaN         DeKalb Ave & Hudson Ave                   NaN   
32952       NaN                 E 16 St & 5 Ave                   NaN   
32953       NaN                 W 44 St & 5 Ave                   NaN   
32954       NaN                 E 56 St & 3 Ave                   NaN   
32955       NaN               E 2 St & Avenue B                   NaN   
32956       NaN                  E 4 St & 2 Ave                   NaN   
32957       NaN              E 17 St & Broadway                   NaN   
32958       NaN                 1 Ave & E 15 St                   NaN   
32959       NaN                 W 51 St & 6 Ave                   NaN   
32960       NaN                 W 11 St & 6 Ave                   NaN   
32961       NaN                 W 20 St & 8 Ave                  

What are average trip duration during weekdays?

In [12]:
query = '''SELECT AVG(tripduration) FROM fb55.citibike
WHERE NOT (EXTRACT(DOW from starttime)=6 OR EXTRACT(DOW from starttime)=0); '''
table = get_data(query)
table

avg  Unnamed: 1
0  681.052292         NaN

Showing the list of start station locations using GROUP BY 

In [13]:
query = '''SELECT start_station_name FROM fb55.citibike
GROUP BY start_station_name; '''
table = get_data(query)
table

start_station_name  Unnamed: 1
0             E 30 St & Park Ave S         NaN
1          Pearl St & Anchorage Pl         NaN
2                  W 56 St & 6 Ave         NaN
3                   West Thames St         NaN
4          Bank St & Washington St         NaN
5                  8 Ave & W 52 St         NaN
6               E 11 St & Broadway         NaN
7         DeKalb Ave & Skillman St         NaN
8    Atlantic Ave & Fort Greene Pl         NaN
9                Avenue D & E 8 St         NaN
10            S 3 St & Bedford Ave         NaN
11          W Broadway & Spring St         NaN
12                 2 Ave & E 31 St         NaN
13                 E 33 St & 5 Ave         NaN
14        MacDougal St & Prince St         NaN
15    Clermont Ave & Lafayette Ave         NaN
16          Barclay St & Church St         NaN
17          Forsyth St & Broome St         NaN
18                 W 13 St & 6 Ave         NaN
19                 W 17 St & 8 Ave         NaN
20              E 20 St & Park Ave         NaN
21               E 2 St & Avenue B         NaN
22                 E 33 St & 2 Ave         NaN
23                 W 18 St & 6 Ave         NaN
24             Old Slip & Front St         NaN
25        Willoughby Ave & Hall St         NaN
26                 1 Ave & E 44 St         NaN
27                 W 20 St & 7 Ave         NaN
28           West St & Chambers St         NaN
29                 E 43 St & 2 Ave         NaN
..                             ...         ...
297               12 Ave & W 40 St         NaN
298        Centre St & Chambers St         NaN
299          Pershing Square South         NaN
300                W 33 St & 7 Ave         NaN
301         Hicks St & Montague St         NaN
302        University Pl & E 14 St         NaN
303        Fulton St & Rockwell Pl         NaN
304    Lafayette Ave & St James Pl         NaN
305        Emerson Pl & Myrtle Ave         NaN
306                E 56 St & 3 Ave         NaN
307        Watts St & Greenwich St         NaN
308                W 44 St & 5 Ave         NaN
309                6 Ave & W 33 St         NaN
310              E 9 St & Avenue C         NaN
311               11 Ave & W 59 St         NaN
312        W 16 St & The High Line         NaN
313          Howard St & Centre St         NaN
314                2 Ave & E 58 St         NaN
315              E 5 St & Avenue C         NaN
316       Fulton St & Clermont Ave         NaN
317    Cadman Plaza E & Tillary St         NaN
318      Pearl St & Hanover Square         NaN
319          LaGuardia Pl & W 3 St         NaN
320                5 Ave & E 29 St         NaN
321                W 21 St & 6 Ave         NaN
322   Wythe Ave & Metropolitan Ave         NaN
323                W 11 St & 6 Ave         NaN
324            St Marks Pl & 1 Ave         NaN
325                W 22 St & 8 Ave         NaN
326         Railroad Ave & Kay Ave         NaN

[327 rows x 2 columns]

Showing the number of trips started per station but only for stations within 500m of Time Square! 

In [14]:
query = '''SELECT
    start_station_name, COUNT(start_station_name), (
      3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )
)*1609.34 AS dist_in_metres
FROM fb55.citibike
WHERE 3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )<0.310686
    GROUP BY start_station_name ,start_station_latitude,start_station_longitude; '''
table = get_data(query)
table

start_station_name  count  dist_in_metres
0     W 43 St & 6 Ave    112      343.910100
1  Broadway & W 41 St    251      294.651915
2     W 42 St & 8 Ave    221      445.410704
3     W 45 St & 6 Ave    141      255.310547
4  Broadway & W 49 St    213      344.795373
5     W 45 St & 8 Ave    141      301.389786
6     W 41 St & 8 Ave    507      391.813077

Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare! 

In [15]:
query = '''SELECT
    start_station_name, AVG(tripduration), (
      3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )
)*1609.34 AS dist
FROM fb55.citibike
WHERE 3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )<0.310686 AND 
   (EXTRACT(DOW from starttime)=6 OR EXTRACT(DOW from starttime)=0)
GROUP BY start_station_name ,start_station_latitude,start_station_longitude
ORDER BY AVG(tripduration) DESC
LIMIT 1; '''
table = get_data(query)
table

start_station_name          avg        dist
0  Broadway & W 49 St  1010.104167  344.795373

Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [28]:
query = '''SELECT
    start_station_name, end_station_name,COUNT(end_station_name),(
      3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )
)*1609.34 AS dist
FROM fb55.citibike
WHERE 3959 * acos (
      cos ( radians(40.7577) )
      * cos( radians( start_station_latitude ) )
      * cos( radians( start_station_longitude ) - radians(-73.9857) )
      + sin ( radians(40.7577) )
      * sin( radians( start_station_latitude ) )
    )<0.310686 AND 
   (tripduration<7200)
   GROUP BY start_station_name ,start_station_latitude,start_station_longitude, end_station_name;'''
table = get_data(query)
table.head()

start_station_name         end_station_name  count        dist
0     W 41 St & 8 Ave          W 45 St & 8 Ave      2  391.813077
1     W 45 St & 8 Ave    E 56 St & Madison Ave      2  301.389786
2     W 42 St & 8 Ave          W 45 St & 6 Ave      1  445.410704
3  Broadway & W 41 St  Lexington Ave & E 24 St      5  294.651915
4     W 42 St & 8 Ave          9 Ave & W 16 St      1  445.410704